In [116]:
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup
from random import randint
import random
from time import sleep
import re
import os
import scipy
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import matplotlib.pyplot as plt
import pickle

In [117]:
big_table = pd.DataFrame()
tables_n = 0
for folder in os.listdir("Clean Data"):
	for i in os.listdir("Clean Data/" + folder):
		table = pd.read_csv("Clean Data/" + folder + "/" + i)
		tables_n += 1
		big_table = pd.concat([table, big_table])
big_table.drop("Unnamed: 0", axis=1, inplace=True)
display(big_table.head())
print(f"Fed by {len(big_table)} lines in {tables_n} tables")

,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Vicenza Calcio,38.0,20.0,7.0,11.0,69.0,45.0,67.0,20
1,2,Atalanta,38.0,17.0,12.0,9.0,51.0,34.0,63.0,20
2,3,Brescia,38.0,16.0,15.0,7.0,54.0,38.0,63.0,20
3,4,Napoli,38.0,17.0,12.0,9.0,55.0,44.0,63.0,20
4,5,Sampdoria,38.0,17.0,11.0,10.0,45.0,40.0,62.0,20


Fed by 5664 lines in 280 tables


In [118]:
big_table.corr()

,rank,GP,W,D,L,GF,GA,P,team_n
rank,1.000000,0.197737,-0.775599,0.178702,0.866332,-0.571336,0.710909,-0.749146,0.197861
GP,0.197737,1.000000,0.318680,0.445955,0.327243,0.381298,0.414891,0.417389,0.999954
W,-0.775599,0.318680,1.000000,-0.158003,-0.619443,0.834402,-0.437760,0.971028,0.318608
D,0.178702,0.445955,-0.158003,1.000000,-0.078370,-0.041819,0.037508,0.057949,0.445728
L,0.866332,0.327243,-0.619443,-0.078370,1.000000,-0.466955,0.819612,-0.639372,0.327424
GF,-0.571336,0.381298,0.834402,-0.041819,-0.466955,1.000000,-0.111340,0.829931,0.381374
GA,0.710909,0.414891,-0.437760,0.037508,0.819612,-0.111340,1.000000,-0.432418,0.415315
P,-0.749146,0.417389,0.971028,0.057949,-0.639372,0.829931,-0.432418,1.000000,0.417270
team_n,0.197861,0.999954,0.318608,0.445728,0.327424,0.381374,0.415315,0.417270,1.000000


In [119]:
X = big_table[["rank", "team_n"]]
y = big_table["P"]

In [120]:
multi_lr = sm.OLS(endog=y, exog=X)
results = multi_lr.fit()

In [128]:
display(results.params)


rank     -2.198652
team_n    3.743901
dtype: float64

In [121]:
results.predict([6,18])[0]

54.19830153630113

In [123]:
wls_prediction_std(results)[0].mean()

5.023467925997238

In [139]:
def average_table(n):
	return round(big_table[big_table["team_n"] == n].groupby("rank").aggregate("mean"),2)

average_table(18)

,GP,W,D,L,GF,GA,P,team_n
rank,,,,,,,,
1,34.00,22.28,6.91,4.81,70.01,29.65,73.76,18.0
2,34.00,19.89,7.93,6.18,64.39,33.53,67.59,18.0
3,34.00,18.23,8.53,7.24,60.57,36.09,63.22,18.0
4,33.99,16.35,8.91,8.73,53.86,37.62,57.91,18.0
5,34.00,15.20,8.68,10.12,51.49,41.74,54.24,18.0
6,34.00,14.19,9.26,10.55,49.85,42.15,51.82,18.0
7,34.00,13.62,8.86,11.51,48.09,44.39,49.68,18.0
8,34.00,12.96,8.54,12.50,46.24,46.68,47.42,18.0
9,34.00,11.97,9.46,12.57,46.72,48.09,45.35,18.0


In [154]:
average_table(19)

,GP,W,D,L,GF,GA,P,team_n
rank,,,,,,,,
1,36.0,23.5,7.0,5.5,85.0,41.0,77.5,19.0
2,36.0,21.5,8.0,6.5,74.0,43.5,72.5,19.0
3,36.0,20.5,6.0,9.5,76.0,45.0,67.5,19.0
4,36.0,19.0,9.0,8.0,65.0,39.5,66.0,19.0
5,36.0,18.5,8.5,9.0,59.5,37.5,64.0,19.0
6,36.0,17.0,9.5,9.5,63.0,47.5,60.5,19.0
7,36.0,16.5,8.0,11.5,71.0,58.0,57.5,19.0
8,36.0,16.5,8.0,11.5,64.0,53.5,57.5,19.0
9,36.0,14.0,10.0,12.0,67.0,55.0,52.0,19.0


In [144]:
round(big_table[big_table["team_n"] == 18].groupby("rank").aggregate("min"),2)

,team_name,GP,W,D,L,GF,GA,P,team_n
rank,,,,,,,,,
1,Ajax,34.0,17.0,1.0,1.0,45.0,16.0,61.0,18
2,Ajax,34.0,15.0,3.0,2.0,39.0,13.0,56.0,18
3,1. FC Union Berlin,34.0,13.0,4.0,2.0,33.0,19.0,54.0,18
4,1. FC Union Berlin,33.0,12.0,3.0,4.0,38.0,22.0,48.0,18
5,1. FC Heidenheim 1846,34.0,12.0,2.0,6.0,36.0,29.0,48.0,18
6,1. FC Heidenheim 1846,34.0,11.0,4.0,5.0,34.0,29.0,45.0,18
7,1. FC Union Berlin,34.0,10.0,4.0,7.0,35.0,32.0,44.0,18
8,1. FC Heidenheim 1846,34.0,10.0,3.0,7.0,28.0,31.0,43.0,18
9,1. FC Union Berlin,34.0,9.0,3.0,9.0,31.0,32.0,41.0,18


In [127]:
with open('model.pkl', 'wb') as f:
	pickle.dump(results, f)

In [151]:
results.score

0.9917577953845145

,rank,team_name,GP,W,D,L,GF,GA,P,team_n
0,1,Vicenza Calcio,38.0,20.0,7.0,11.0,69.0,45.0,67.0,20
1,2,Atalanta,38.0,17.0,12.0,9.0,51.0,34.0,63.0,20
2,3,Brescia,38.0,16.0,15.0,7.0,54.0,38.0,63.0,20
3,4,Napoli,38.0,17.0,12.0,9.0,55.0,44.0,63.0,20
4,5,Sampdoria,38.0,17.0,11.0,10.0,45.0,40.0,62.0,20
...,...,...,...,...,...,...,...,...,...,...
13,14,Dynamo Dresden,34.0,11.0,8.0,15.0,42.0,52.0,41.0,18
14,15,SpVgg Greuther Fürth,34.0,10.0,10.0,14.0,37.0,48.0,40.0,18
15,16,Erzgebirge Aue,34.0,10.0,10.0,14.0,35.0,49.0,40.0,18
16,17,Eintracht Braunschweig,34.0,8.0,15.0,11.0,37.0,43.0,39.0,18
